In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pyspark.mllib.evaluation import RegressionMetrics

class RecurrentNeuralNet:    
  def __init__(self, inputFile, trainSplit, learningRate, numEpochs, hiddenDimensions, backPropTruncate, minClip, maxClip, timeSteps, activationFunction):
    #load input file
    inputData = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load(inputFile)
    
    #save parameters for RNN for use in program
    self.trainSplit = trainSplit
    self.learningRate = learningRate
    self.numEpochs = numEpochs
    self.hiddenDimensions = hiddenDimensions
    self.outputDimension = 1
    self.backPropTruncate = backPropTruncate
    self.minClip = minClip
    self.maxClip = maxClip
    self.timeSteps = timeSteps
    self.activationFunction  = activationFunction
    
    #get size of train and test data based on split
    trainSize = int(inputData.count() * self.trainSplit)
    testSize = inputData.count()- trainSize
    
    #preprocessing to scale data
    inputData = inputData.toPandas()
    self.closeScaler =MinMaxScaler(feature_range=(0,1))
    inputData[['Close']] = self.closeScaler.fit_transform(inputData.loc[:, ['Close']])
    closingStockPrice = inputData['Close'].tolist()

    #split into test and train set   
    self.train = closingStockPrice[:trainSize]
    self.test = closingStockPrice[-testSize:]
    
    #generate sequences of data to train on for train and test
    self.inputTrain, self.outputTrain = self.generateSeqs(self.train, self.timeSteps)
    self.inputTest, self.outputTest = self.generateSeqs(self.test, self.timeSteps)
    
    #generate random starting between 0 and 1 
    self.U= np.random.rand(hiddenDimensions, timeSteps)
    self.W = np.random.rand(hiddenDimensions, hiddenDimensions)
    self.V = np.random.rand(self.outputDimension, hiddenDimensions)

  #generate sequences of input and output data to perform training and testing on 
  def generateSeqs(self, input, steps):
    inputData = []
    targetData = []
    for x in range (len(input) - steps):
      #get series of data
      inputData.append(input[x: x + steps])
      #get target value to predict
      targetData.append(input[x + steps])
    return np.expand_dims(np.array(inputData), axis = 2), np.expand_dims(np.array(targetData), axis=1)
  
  
  #define activation functions and derivatives
  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  def sigmoidDerivative(self,x):
    return x*(1-x)

  def tanh(self, x):
    return np.tanh(x)

  def tanhDerivative(self, x):
    return 1-x**2
  
  def activation(self, x):
    if self.activationFunction == "sigmoid":
      return self.sigmoid(x)
    elif self.activationFunction == "tanh":
      return self.tanh(x)
  
  def activationDerivative(self, x):
    if self.activationFunction == "sigmoid":
      return self.sigmoidDerivative(x)
    elif self.activationFunction == "tanh":
      return self.tanhDerivative(x)
  

  #simple forward pass
  def forward(self, numSequences, X, Y):
    loss = 0
    for i in range(numSequences):
      x, y = X[i], Y[i]                    
      prevHidden = np.zeros((self.hiddenDimensions, 1))  
      for t in range(self.timeSteps):
          input = np.zeros(x.shape)    
          input[t] = x[t]             
          hiddenState = self.activation(np.dot(self.U, input) + np.dot(self.W, prevHidden))
          outputState = np.dot(self.V, hiddenState)
          prevHidden = hiddenState
      loss += (y - outputState)**2
    loss = loss/float(self.outputTrain.shape[0])  
    return x, y, outputState, loss
   
  #forward pass keeping track of layers between timesteps
  def forwardWithLayers(self, x):
    prevHidden = np.zeros((hiddenDimensions, 1))
    self.layers = []
    for t in range(self.timeSteps):
      input = np.zeros(x.shape)
      input[t] = x[t]
      sum = np.dot(self.U, input) + np.dot(self.W, prevHidden)
      hiddenState = self.activation(sum)
      outputState = np.dot(self.V, hiddenState)
      self.layers.append({'hiddenState':hiddenState, 'prevHidden':prevHidden})
      prevHidden = hiddenState
    return x, sum, outputState, prevHidden
  
  def trainModel(self):
    for i in range(self.outputTrain.shape[0]):
      x, y = self.inputTrain[i], self.outputTrain[i]
      #create variables to hold gradient values
      dU = np.zeros(self.U.shape)
      dV = np.zeros(self.V.shape)
      dW = np.zeros(self.W.shape)
      
      #create vars to help calculate gradient
      dUtime = np.zeros(self.U.shape)
      dVtime = np.zeros(self.V.shape)
      dWtime = np.zeros(self.W.shape)
      
      #forward pass
      x, sum, outputState, prevHidden = self.forwardWithLayers(x)
    dOut = (outputState-y)
  
    #backward pass
    for t in range(self.timeSteps):
      dVtime = np.dot(dOut, np.transpose(self.layers[t]['hiddenState']))
      dOutStep = np.dot(np.transpose(self.V), dOut)
      dSumAct = self.activationDerivative(sum) * dOutStep
      dPrevHidden = np.dot(np.transpose(self.W), dSumAct * np.ones_like((self.hiddenDimensions, 1)))
      #backprop through time
      for i in reversed(range(t-1, t-backPropTruncate-1)):
        dSumAct = self.activationDerivative(sum) * (dOutStep + dPrevHidden)
        dWiterTime = np.dot(self.W, self.layers[t]['prevHidden'])
        dPrevHidden = np.dot(np.transpose(self.W), dSumAct * np.ones_like((self.hiddenDimensions, 1)))
        input = np.zeros(x.shape)
        input[t] = x[t]

        dUtime += np.dot(self.U, input)
        dWtime += np.dot(self.W, self.layers[t]['prevHidden'])
        
      #update gradient weights  
      dV += dVtime
      dU += dUtime
      dW += dWtime
      
      #perform gradient clipping to prevent exploding gradient
      dU = dU.clip(minClip, maxClip)
      dV = dV.clip(minClip, maxClip)
      dW = dW.clip(minClip, maxClip)
      
    #update weights
    self.U -= self.learningRate * dU
    self.V -= self.learningRate * dV
    self.W -= self.learningRate * dW   

  def fitModel(self):
    for e in range(self.numEpochs):
      # check loss on training data
      x, y, outputState, trainLoss = self.forward(self.outputTrain.shape[0], self.inputTrain, self.outputTrain)
      # check loss based on test set
      x, y, outputState, testLoss= self.forward(self.outputTest.shape[0], self.inputTest, self.outputTest )
      print('Epoch: ', e + 1, ', Training Loss: ', trainLoss, ', Test Loss: ', testLoss)    
      self.trainModel()
      
  def makePredictions(self, numSequences, input):
    preds = []
    for i in range(numSequences):
        x = input[i]
        prevHidden = np.zeros((self.hiddenDimensions, 1))
        for t in range(self.timeSteps):
            hiddenState = self.activation(np.dot(self.U, x) + np.dot(self.W, prevHidden))
            outputState = np.dot(self.V, hiddenState)
            prevHidden = hiddenState
        preds.append(outputState)
    preds = np.array(preds)
    reshapePreds = pd.DataFrame.from_records(preds.reshape(-1,1))
    return self.closeScaler.inverse_transform(reshapePreds)

  def evaluate(self):   
    trainPredictions = self.makePredictions(self.outputTrain.shape[0], self.inputTrain)
    trainActual = self.closeScaler.inverse_transform(self.outputTrain)
    trainMetrics = self.calculateRMSE(trainPredictions, trainActual)
    print("Training RMSE = %s" % trainMetrics)
    
    testPredictions = self.makePredictions(self.outputTest.shape[0], self.inputTest)
    testActual = self.closeScaler.inverse_transform(self.outputTest)
    testMetrics = self.calculateRMSE(testPredictions, testActual)
    print("Test RMSE = %s" % testMetrics)

  
  def calculateRMSE(self, predictions, actual):
    predictionsList  = predictions.ravel().tolist()
    actualList = actual.ravel().tolist()
    mergeLists = list(zip(predictionsList, actualList))
    predictionsAndActual = sc.parallelize(mergeLists)
    predictionsAndActual.collect()
    metrics = RegressionMetrics(predictionsAndActual)
    return metrics.rootMeanSquaredError
    


 

In [0]:
 %sh 
 curl -O https://s3.amazonaws.com/cs6350.0u1/Project/infy_stock.csv

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 30177 100 30177 0 0 58378 0 --:--:-- --:--:-- --:--:-- 58482

In [0]:
inputFile = "file:/databricks/driver/infy_stock.csv"

#Params that can be changed
trainSplit = 0.8
earningRate = 0.001
numEpochs =  100
hiddenDimensions = 100
backPropTruncate = 5
minClip = -10
maxClip = 10
timeSteps = 15;
activationFunction = "tanh"

rnn = RecurrentNeuralNet(inputFile, trainSplit, learningRate, numEpochs, hiddenDimensions, backPropTruncate, minClip, maxClip, timeSteps, activationFunction)
rnn.fitModel()
rnn.evaluate()


Epoch: 1 , Training Loss: [[2550.91185813]] , Test Loss: [[495.63981602]]
Epoch: 2 , Training Loss: [[2450.90176236]] , Test Loss: [[476.35858063]]
Epoch: 3 , Training Loss: [[2352.8916666]] , Test Loss: [[457.4598589]]
Epoch: 4 , Training Loss: [[2256.88157083]] , Test Loss: [[438.94365083]]
Epoch: 5 , Training Loss: [[2162.87147507]] , Test Loss: [[420.80995642]]
Epoch: 6 , Training Loss: [[2070.8613793]] , Test Loss: [[403.05877568]]
Epoch: 7 , Training Loss: [[1980.85128353]] , Test Loss: [[385.69010859]]
Epoch: 8 , Training Loss: [[1892.84118777]] , Test Loss: [[368.70395516]]
Epoch: 9 , Training Loss: [[1806.831092]] , Test Loss: [[352.1003154]]
Epoch: 10 , Training Loss: [[1722.82099624]] , Test Loss: [[335.8791893]]
Epoch: 11 , Training Loss: [[1640.81090047]] , Test Loss: [[320.04057686]]
Epoch: 12 , Training Loss: [[1560.80080471]] , Test Loss: [[304.58447808]]
Epoch: 13 , Training Loss: [[1482.79070894]] , Test Loss: [[289.51089296]]
Epoch: 14 , Training Loss: [[1406.78061318]] , Test Loss: [[274.8198215]]
Epoch: 15 , Training Loss: [[1332.77051741]] , Test Loss: [[260.5112637]]
Epoch: 16 , Training Loss: [[1260.76042164]] , Test Loss: [[246.58521957]]
Epoch: 17 , Training Loss: [[1190.75032588]] , Test Loss: [[233.04168909]]
Epoch: 18 , Training Loss: [[1122.74023011]] , Test Loss: [[219.88067228]]
Epoch: 19 , Training Loss: [[1056.73013435]] , Test Loss: [[207.10216913]]
Epoch: 20 , Training Loss: [[992.72003858]] , Test Loss: [[194.70617964]]
Epoch: 21 , Training Loss: [[930.70994282]] , Test Loss: [[182.69270381]]
Epoch: 22 , Training Loss: [[870.69984705]] , Test Loss: [[171.06174164]]
Epoch: 23 , Training Loss: [[812.68975128]] , Test Loss: [[159.81329313]]
Epoch: 24 , Training Loss: [[756.67965552]] , Test Loss: [[148.94735828]]
Epoch: 25 , Training Loss: [[702.66955975]] , Test Loss: [[138.4639371]]
Epoch: 26 , Training Loss: [[650.65946399]] , Test Loss: [[128.36302958]]
Epoch: 27 , Training Loss: [[600.64936822]] , Test Loss: [[118.64463571]]
Epoch: 28 , Training Loss: [[552.63927246]] , Test Loss: [[109.30875551]]
Epoch: 29 , Training Loss: [[506.62917669]] , Test Loss: [[100.35538897]]
Epoch: 30 , Training Loss: [[462.61908093]] , Test Loss: [[91.78453609]]
Epoch: 31 , Training Loss: [[420.60898516]] , Test Loss: [[83.59619688]]
Epoch: 32 , Training Loss: [[380.59888939]] , Test Loss: [[75.79037132]]
Epoch: 33 , Training Loss: [[342.58879363]] , Test Loss: [[68.36705943]]
Epoch: 34 , Training Loss: [[306.57869786]] , Test Loss: [[61.32626119]]
Epoch: 35 , Training Loss: [[272.5686021]] , Test Loss: [[54.66797662]]
Epoch: 36 , Training Loss: [[240.55850633]] , Test Loss: [[48.39220571]]
Epoch: 37 , Training Loss: [[210.54841057]] , Test Loss: [[42.49894846]]
Epoch: 38 , Training Loss: [[182.5383148]] , Test Loss: [[36.98820487]]
Epoch: 39 , Training Loss: [[156.52821904]] , Test Loss: [[31.85997494]]
Epoch: 40 , Training Loss: [[132.51812327]] , Test Loss: [[27.11425867]]
Epoch: 41 , Training Loss: [[110.5080275]] , Test Loss: [[22.75105607]]
Epoch: 42 , Training Loss: [[90.49793174]] , Test Loss: [[18.77036712]]
Epoch: 43 , Training Loss: [[72.48783597]] , Test Loss: [[15.17219184]]
Epoch: 44 , Training Loss: [[56.47774021]] , Test Loss: [[11.95653022]]
Epoch: 45 , Training Loss: [[42.46764444]] , Test Loss: [[9.12338226]]
Epoch: 46 , Training Loss: [[30.45754868]] , Test Loss: [[6.67274796]]
Epoch: 47 , Training Loss: [[20.44745291]] , Test Loss: [[4.60462732]]
Epoch: 48 , Training Loss: [[12.43735715]] , Test Loss: [[2.91902034]]
Epoch: 49 , Training Loss: [[6.42726138]] , Test Loss: [[1.61592703]]
Epoch: 50 , Training Loss: [[2.41716561]] , Test Loss: [[0.69534737]]
Epoch: 51 , Training Loss: [[0.40706985]] , Test Loss: [[0.15728138]]
Epoch: 52 , Training Loss: [[0.39697408]] , Test Loss: [[0.00172905]]
Epoch: 53 , Training Loss: [[0.26872259]] , Test Loss: [[0.00075026]]
Epoch: 54 , Training Loss: [[0.31513106]] , Test Loss: [[6.36885164e-05]]
Epoch: 55 , Training Loss: [[0.29535252]] , Test Loss: [[